In [ ]:
!pip install streamlit py3Dmol biopython torch transformers ngrok


In [ ]:
import torch
from transformers import EsmForProteinFolding, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("facebook/esmfold_v1")
model = EsmForProteinFolding.from_pretrained("facebook/esmfold_v1")
model.eval()

def sequence_to_pdb(sequence):
    inputs = tokenizer(sequence, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    pdb = model.output_to_pdb(outputs)[0]
    return pdb


In [ ]:
import py3Dmol

def show_structure(pdb_string):
    view = py3Dmol.view(width=400, height=400)
    view.addModel(pdb_string, "pdb")
    view.setStyle({"cartoon": {"color": "spectrum"}})
    view.zoomTo()
    return view


In [ ]:
import streamlit as st
from Bio.SeqUtils import molecular_weight
import py3Dmol

st.set_page_config(layout="wide")

st.title("Protein Sequence & Structure Viewer")

sequence = st.text_area(
    "Enter Protein Sequence (FASTA)",
    "MVKVGVNGFGRIGRLVTRAAFNSGKVDIVAINDPFIDLNYMVYMFQYDSTHGKFHGTVKAENGKLVINGNPITIFQERDPANLPWGK"
)

if st.button("Predict Structure"):
    with st.spinner("Predicting protein structure..."):
        pdb = sequence_to_pdb(sequence)

    col1, col2 = st.columns(2)

    with col1:
        st.subheader("Protein Sequence")
        st.text(sequence)
        st.write("Length:", len(sequence))
        st.write("Molecular Weight:", molecular_weight(sequence, "protein"))

    with col2:
        st.subheader("3D Protein Structure")
        view = py3Dmol.view(width=500, height=500)
        view.addModel(pdb, "pdb")
        view.setStyle({"cartoon": {"color": "spectrum"}})
        view.zoomTo()
        st.components.v1.html(view._make_html(), height=550)
